In [2]:
import cv2 as cv
import numpy as np
from IPython.display import Image, display
import math
from numpy.linalg import norm
import random
import PIL.Image
import io
import json
import pickle
def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
from skspatial.objects import Line
from sklearn.cluster import MeanShift, estimate_bandwidth

In [59]:
def check_intersection(line1, line2):
    line1_sk = Line.from_points(line1[:2], line1[2:])
    line2_sk = Line.from_points(line2[:2], line2[2:])

    try:
        intersect = line1_sk.intersect_line(line2_sk)
    except ValueError:
        return False, None

    return True, intersect

def cos_similarity(line1, line2):
    A = np.array(line1[:2] - line1[2:])
    B = np.array(line2[:2] - line2[2:])
    return np.dot(A,B)/(norm(A) * norm(B))

def find_grid_points(frame):
    # Canny edge and hough lines
    edges = cv.Canny(frame,50,250)
    edges = cv.dilate(edges.astype('uint8'), np.ones((2,2), np.uint8), iterations=3)

    lines = cv.HoughLinesP(edges, 2, math.radians(0.1), 400, minLineLength=200, maxLineGap=20)
    if lines is None:
        return [], frame
    for line in lines:
        x1,y1,x2,y2 = line[0]
        cv.line(frame,(x1,y1),(x2,y2),(0,255,0),2)

    # Find points intersecting with line
    points = []
    for i,l1 in enumerate(lines):
        for j,l2 in enumerate(lines):
            if i == j:
                continue
    
            similarity = cos_similarity(l1[0], l2[0])
            #print(l1[0][2])
            has_intersect, point = check_intersection(l1[0].astype(float), l2[0].astype(float))
            if has_intersect and similarity < 0.6:
                oob = np.any(point > frame.shape[:2]) | np.any(point < 0)
                if not oob:
                    points.append(point)

    for point in points:
        cv.circle(frame, center=(point[0].astype('uint32'), point[1].astype('uint32')), radius=2, color=(255,0,0))

    if len(points) == 0:
        return [], frame
    bandwidth = estimate_bandwidth(points, quantile=0.025, n_samples=50000)
    if bandwidth == 0:
        return [], frame
    clustering = MeanShift(bandwidth=bandwidth).fit(points)
    centers = clustering.cluster_centers_
    cluster_frame = frame.copy()
    for i, point in enumerate(centers):
        location = (point[0].astype('uint32'), point[1].astype('uint32'))
        cv.circle(frame, center=location, thickness=-1, radius=6, color=(0,100,255))
        frame = cv.putText(frame, f"{i}", location, cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,20,255), 2, cv.LINE_AA)

    return centers, frame

In [3]:
# Read all of video into a large list (bye bye memory)
cap = cv.VideoCapture("patitos.mp4")

ret, frame = cap.read()
frames = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frames.append(frame)

In [4]:
len(frames)

2082

In [60]:
# Draw samples
n_frames = 50
samples = random.choices(frames, k=n_frames)

# Export samples and json file with data
centers, annotated_frames = [], []
for s in samples:
    c, a_f = find_grid_points(s)
    centers.append(c)
    annotated_frames.append(a_f)

path = "camera_calibration"
# Export annotated frames to folder
for i,frame in enumerate(annotated_frames):
    cv.imwrite(f"{path}/{i}.jpg", frame)

data_out = []
point_data = []
for i,c in enumerate(centers):
    data_out.append({
        "frame_id": i,
        "centers": c,
    })

    point_data.append({
        "frame_id": i,
        "points": [0],
        "n_rows": 0,
        "n_cols" : 0})

json_obj = json.dumps(point_data, indent=2)
with open(f"{path}/point_data.json", "w") as outfile:
    outfile.write(json_obj)

center_obj = pickle.dumps(data_out)
with open(f"{path}/center.pkl", "wb") as outfile:
    outfile.write(center_obj)


In [48]:
frame.shape

(480, 848, 3)

In [28]:
cap = cv.VideoCapture("patitos.mp4")

ret, frame = cap.read()
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

video = cv.VideoWriter('outpy.avi',cv.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

frame_count = 0
while True:
    frame_count += 1
    ret, frame = cap.read()
    #print(frame_count)
    if not ret or frame_count > 500:
        break
    centers, frame = find_grid_points(frame)
    video.write(frame)

video.release()

In [14]:
frame.shape[:2]

(480, 848)